### Consulta 1
def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

-- precio en tabla steam_games
-- recommend (True or false) en tabla user_revs
-- cantidad de items sería un count de unique (item_id) en tabla user_items

In [50]:
import pandas as pd

In [51]:
columns = ['id', 'price']
games_price = pd.read_csv(r'./datasets/steam_games.csv', usecols=columns)
games_price.head()

,price,id
0,4.99,761140.0
1,Free To Play,643980.0
2,Free to Play,670290.0
3,0.99,767400.0
4,2.99,773570.0


In [52]:
games_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   30758 non-null  object 
 1   id      32133 non-null  float64
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


In [53]:
unique_list = []
for i in games_price['price']:
    try:
        float(i)
    except:
        unique_list.append(i)

uniques = set(unique_list)
uniques_not_free = ['Starting at $499.00', 'Starting at $449.00']

In [54]:
print(uniques)

{'Free to Try', 'Free to Play', 'Play for Free!', 'Play WARMACHINE: Tactics Demo', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Free To Play', 'Play the Demo', 'Starting at $499.00', 'Third-party', 'Free Movie', 'Starting at $449.00', 'Free', 'Install Now', 'Free Demo', 'Install Theme', 'Free to Use', 'Free Mod'}


In [55]:
games_price['price_fixed'] = games_price['price'].apply(lambda x: 0 if x in uniques 
                                                        else 499.0 if x=='Starting at $499.00'
                                                        else 449.0 if x=='Starting at $449.00'
                                                        else x)
games_price.head()

,price,id,price_fixed
0,4.99,761140.0,4.99
1,Free To Play,643980.0,0
2,Free to Play,670290.0,0
3,0.99,767400.0,0.99
4,2.99,773570.0,2.99


In [56]:
games_price['price_fixed'] = games_price['price_fixed'].astype(float)

In [57]:
games_price.rename(columns={'id':'item_id'},inplace=True)
games_price.drop(columns='price', inplace=True)
games_price.head()

,item_id,price_fixed
0,761140.0,4.99
1,643980.0,0.00
2,670290.0,0.00
3,767400.0,0.99
4,773570.0,2.99


In [58]:
games_price.isna().sum()

item_id           2
price_fixed    1377
dtype: int64

In [59]:
games_price.fillna(0, inplace=True)

In [60]:
games_price.isna().sum()

item_id        0
price_fixed    0
dtype: int64

In [61]:
columns = ['steam_id', 'item_id', 'items_count']
user_items = pd.read_csv('datasets/aus_users_items.csv', usecols=columns)
user_items.head()

,items_count,steam_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,20
2,277,76561197970982479,30
3,277,76561197970982479,40
4,277,76561197970982479,50


In [62]:
df_merged = user_items.merge(games_price, on='item_id', how='left')
df_merged.sample(10)

,items_count,steam_id,item_id,price_fixed
2863209,96,76561198049136740,261030,24.99
475095,44,76561198089673624,442710,0.00
2611937,65,76561198033081099,250420,0.00
294037,46,76561198151185646,353360,0.00
112978,480,76561198098805637,32400,5.99
1282394,155,76561198018842433,252950,19.99
1977985,160,76561198042240326,6900,6.99
4177113,62,76561198033159870,230410,0.00
888887,160,76561197960561678,253710,0.00
2814571,250,76561198032558761,12150,9.99


In [64]:
user_spent = df_merged.groupby('steam_id')['price_fixed'].agg('sum')
user_spent

steam_id
76561197960269200    4768.22
76561197960270504    2788.45
76561197960270742     340.73
76561197960271609     239.80
76561197960271888    1633.80
                      ...   
76561198327778018      19.99
76561198328421116      74.98
76561198328629223       0.00
76561198328946162      19.98
76561198329548331       0.00
Name: price_fixed, Length: 70912, dtype: float64

In [66]:
columns = ['steam_id', 'recommend']
user_revs = pd.read_csv('datasets/aus_user_revs.csv', usecols=columns)
user_revs.sample(5)


/var/folders/g_/fkn0ttqn6w5_k96kvk3ctwm80000gn/T/ipykernel_16537/580802193.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  user_revs = pd.read_csv('datasets/aus_user_revs.csv', usecols=columns)


,steam_id,recommend
62993,76561198331598578,True
152416,+ I really enjoy the sound system; how the zom...,NaN
60251,76561198331598578,True
138808,+ I really enjoy the sound system; how the zom...,NaN
184140,+ I really enjoy the sound system; how the zom...,NaN


In [67]:
user_revs.loc[152416]

steam_id     + I really enjoy the sound system; how the zom...
recommend                                                  NaN
Name: 152416, dtype: object

In [68]:
user_revs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263226 entries, 0 to 263225
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   steam_id   263226 non-null  object
 1   recommend  66616 non-null   object
dtypes: object(2)
memory usage: 4.0+ MB


In [89]:
unique_list = []
for i,x in enumerate(user_revs['steam_id']):
    try:
        long(x)
    except:
        unique_list.append(i)

#uniques = set(unique_list)
len(unique_list)

263226

In [90]:
unique_list = []
for x in (user_revs['steam_id']):
    if len(x) > 50:
        unique_list.append(x)

#print(unique_list)
print(len(unique_list))

196610
